In [2]:
import json
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from datetime import datetime
from google.colab import files
uploaded = files.upload()

Saving user-wallet-transactions.json to user-wallet-transactions.json


In [3]:
with open(next(iter(uploaded))) as f:
    raw_data = json.load(f)

print(f"Loaded {len(raw_data)} transactions")

def get_usd_value(entry):
    try:
        return float(entry["actionData"]["amount"]) * float(entry["actionData"]["assetPriceUSD"])
    except:
        return 0.0

wallets = defaultdict(lambda: {
    "num_txns": 0,
    "total_deposited": 0.0,
    "total_borrowed": 0.0,
    "total_repaid": 0.0,
    "total_redeemed": 0.0,
    "num_liquidations": 0,
    "first_ts": float('inf'),
    "last_ts": 0,
})

for entry in tqdm(raw_data):
    wallet = entry["userWallet"]
    ts = entry["timestamp"]
    action = entry["action"].lower()

    wallets[wallet]["num_txns"] += 1
    wallets[wallet]["first_ts"] = min(wallets[wallet]["first_ts"], ts)
    wallets[wallet]["last_ts"] = max(wallets[wallet]["last_ts"], ts)

    usd_val = get_usd_value(entry)

    if action == "deposit":
        wallets[wallet]["total_deposited"] += usd_val
    elif action == "borrow":
        wallets[wallet]["total_borrowed"] += usd_val
    elif action == "repay":
        wallets[wallet]["total_repaid"] += usd_val
    elif action == "redeemunderlying":
        wallets[wallet]["total_redeemed"] += usd_val
    elif action == "liquidationcall":
        wallets[wallet]["num_liquidations"] += 1

results = []

for wallet, data in wallets.items():
    repay_ratio = min(data["total_repaid"] / data["total_borrowed"], 1) if data["total_borrowed"] > 0 else 1
    redeem_ratio = min(data["total_redeemed"] / data["total_deposited"], 1) if data["total_deposited"] > 0 else 1
    borrow_deposit_ratio = data["total_borrowed"] / data["total_deposited"] if data["total_deposited"] > 0 else 0

    span_days = max(1, (data["last_ts"] - data["first_ts"]) // 86400)
    avg_txn_per_day = data["num_txns"] / span_days
    score = 500
    if repay_ratio > 0.9: score += 100
    if redeem_ratio > 0.8: score += 100
    if span_days > 90: score += 100
    if avg_txn_per_day > 1: score += 50
    if borrow_deposit_ratio > 1.5: score -= 100

    score -= data["num_liquidations"] * 100
    score = max(0, min(1000, round(score)))

    results.append({
        "wallet": wallet,
        "score": score,
        "txns": data["num_txns"],
        "repay_ratio": round(repay_ratio, 3),
        "redeem_ratio": round(redeem_ratio, 3),
        "borrow_deposit_ratio": round(borrow_deposit_ratio, 3),
        "avg_txns_per_day": round(avg_txn_per_day, 3),
        "num_liquidations": data["num_liquidations"]
    })
df_scores = pd.DataFrame(results).sort_values(by="score", ascending=False)
df_scores.head(10)


Loaded 100000 transactions


100%|██████████| 100000/100000 [00:00<00:00, 314892.42it/s]


,wallet,score,txns,repay_ratio,redeem_ratio,borrow_deposit_ratio,avg_txns_per_day,num_liquidations
2424,0x042a1ccde458f357618d8687b4fb99fa10775599,850,206,0.958,0.971,0.606,1.807,0
2648,0x049940feda4277b7f01ef10fca0b975c541d8fca,850,345,1.000,1.000,0.558,3.136,0
912,0x017f9dc0e2d0a123bf03a3c7ec27f898b70d8ded,850,141,1.000,1.000,0.008,1.439,0
2973,0x052f348a45620f2c26926d2b58548c7eaeadf870,850,166,1.000,1.000,0.039,1.566,0
2185,0x03b16ab6e23bdbeeab719d8e4c49d63674876253,850,597,0.999,1.000,0.229,4.234,0
463,0x00b43a92d45ccdfeef6ea98d56f775586fe9e39e,850,189,1.000,1.000,0.058,1.500,0
3019,0x05404b6f8990a4108150366adb572a870b137edc,850,473,0.974,0.947,0.147,3.504,0
1763,0x02faf4316dcb9330751f3677436aa849aa93851b,850,248,1.000,1.000,0.821,2.120,0
777,0x0144c47df1cc4d52ff115f758918dc42a9db48aa,850,293,1.000,0.843,0.068,2.363,0
3405,0x05ebf7b089a859af295b4f4d96673edcb5ef6621,850,303,1.000,1.000,0.400,2.278,0


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler

features = ["txns", "repay_ratio", "redeem_ratio", "borrow_deposit_ratio", "avg_txns_per_day", "num_liquidations"]
X = df_scores.drop(columns=["wallet","score"]).values
y = df_scores["score"].values.reshape(-1, 1)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

joblib.dump(scaler_X, "scaler_X.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [5]:
class WalletNet(nn.Module):
    def __init__(self, input_size):
        super(WalletNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [6]:
epochs = 500
model = WalletNet(input_size=len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if epoch % 20 == 0:
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch}: Avg Loss = {avg_loss:.6f}")

torch.save(model.state_dict(), 'wallet_net.pth')

Epoch 0: Avg Loss = 0.483754
Epoch 20: Avg Loss = 0.001856
Epoch 40: Avg Loss = 0.001355
Epoch 60: Avg Loss = 0.001176
Epoch 80: Avg Loss = 0.001094
Epoch 100: Avg Loss = 0.001082
Epoch 120: Avg Loss = 0.001038
Epoch 140: Avg Loss = 0.001038
Epoch 160: Avg Loss = 0.001030
Epoch 180: Avg Loss = 0.000975
Epoch 200: Avg Loss = 0.000972
Epoch 220: Avg Loss = 0.000957
Epoch 240: Avg Loss = 0.000953
Epoch 260: Avg Loss = 0.000964
Epoch 280: Avg Loss = 0.000959
Epoch 300: Avg Loss = 0.000984
Epoch 320: Avg Loss = 0.000894
Epoch 340: Avg Loss = 0.000925
Epoch 360: Avg Loss = 0.000909
Epoch 380: Avg Loss = 0.000900
Epoch 400: Avg Loss = 0.000868
Epoch 420: Avg Loss = 0.000880
Epoch 440: Avg Loss = 0.000828
Epoch 460: Avg Loss = 0.000849
Epoch 480: Avg Loss = 0.000782


In [7]:
model.eval()
with torch.no_grad():
    total_loss = 0.0
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        total_loss += loss.item()
    avg_eval_loss = total_loss / len(train_loader)
    print(f"Evaluation Loss on Training Set: {avg_eval_loss:.6f}")

with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor).item()
    print(f"Test Loss: {test_loss:.6f}")

predicted_scores = scaler_y.inverse_transform(test_outputs.detach().numpy())
true_scores = scaler_y.inverse_transform(y_test_tensor.detach().numpy())

for i in range(5):
    print(f"True: {true_scores[i][0]:.2f}, Predicted: {predicted_scores[i][0]:.2f}")


Evaluation Loss on Training Set: 0.000745
Test Loss: 0.000874
True: 600.00, Predicted: 599.32
True: 600.00, Predicted: 599.32
True: 600.00, Predicted: 599.32
True: 600.00, Predicted: 595.41
True: 750.00, Predicted: 730.57


In [8]:
import json
import torch
import torch.nn as nn
import joblib
from collections import defaultdict

class WalletNet(nn.Module):
    def __init__(self, input_size):
        super(WalletNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

input_size = 6
model = WalletNet(input_size)
model.load_state_dict(torch.load("wallet_net.pth"))
model.eval()
scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")

def extract_features(txns):
    deposit, borrow, repay, redeem = 0, 0, 0, 0
    total_txns = len(txns)
    first_ts = float('inf')
    last_ts = 0
    liquidations = 0

    for txn in txns:
        ts = txn.get("timestamp", 0)
        first_ts = min(first_ts, ts)
        last_ts = max(last_ts, ts)

        action = txn.get("action", "").lower()

        try:
            amt = float(txn["actionData"]["amount"])
            price = float(txn["actionData"]["assetPriceUSD"])
            usd_val = amt * price
        except:
            usd_val = 0

        if action == "deposit":
            deposit += usd_val
        elif action == "borrow":
            borrow += usd_val
        elif action == "repay":
            repay += usd_val
        elif action == "redeemunderlying":
            redeem += usd_val
        elif action == "liquidationcall":
            liquidations += 1

    repay_ratio = min(repay / borrow, 1) if borrow > 0 else 1
    redeem_ratio = min(redeem / deposit, 1) if deposit > 0 else 1
    borrow_deposit_ratio = borrow / deposit if deposit > 0 else 0
    span_days = max(1, (last_ts - first_ts) // 86400)
    avg_txns_per_day = total_txns / span_days

    return [
        total_txns,
        repay_ratio,
        redeem_ratio,
        borrow_deposit_ratio,
        avg_txns_per_day,
        liquidations
    ]

def prepare_input_json(raw_file, output_file="input.json"):
    with open(raw_file) as f:
        txns = json.load(f)

    wallets = defaultdict(list)
    for txn in txns:
        wallets[txn["userWallet"].lower()].append(txn)

    grouped = {
        "wallets": [
            {"wallet": wallet, "transactions": txns}
            for wallet, txns in wallets.items()
        ]
    }

    with open(output_file, "w") as f:
        json.dump(grouped, f, indent=2)
    print(f"✅ Saved grouped input to {output_file}")

def predict_credit_scores(model, wallet_entries, scaler_X, scaler_y):
    model.eval()
    output_data = {}

    for entry in wallet_entries:
        wallet_addr = entry["wallet"]
        txns = entry["transactions"]

        features = extract_features(txns)
        if features is None:
            print(f"Skipping wallet {wallet_addr} due to missing features.")
            continue

        features_scaled = scaler_X.transform([features])
        tensor_input = torch.tensor(features_scaled, dtype=torch.float32)

        with torch.no_grad():
            raw_score = model(tensor_input).item()
            print(f"{wallet_addr} → raw model output (scaled): {raw_score:.4f}")

            score = scaler_y.inverse_transform([[raw_score]])[0][0]
            score = max(300, min(850, round(score)))

        output_data[wallet_addr] = score

    with open("output.json", "w") as f:
        json.dump(output_data, f, indent=4)

    print("✅ Prediction complete. Results saved to output.json.")

if __name__ == "__main__":
    prepare_input_json("user-wallet-transactions.json")  # <== your raw file
    with open("input.json") as f:
        data = json.load(f)
        wallet_entries = data["wallets"]
    predict_credit_scores(model, wallet_entries, scaler_X, scaler_y)
    print("✅ Credit scores written to output.json")


✅ Saved grouped input to input.json
0x00000000001accfa9cef68cf5371a23025b6d4b6 → raw model output (scaled): 0.7051
0x000000000051d07a4fb3bd10121a343d85818da6 → raw model output (scaled): 0.7051
0x000000000096026fb41fc39f9875d164bd82e2dc → raw model output (scaled): 0.7005
0x0000000000e189dd664b9ab08a33c4839953852c → raw model output (scaled): 0.8310
0x0000000002032370b971dabd36d72f3e5a7bf1ee → raw model output (scaled): 0.7567
0x000000000a38444e0a6e37d3b630d7e855a7cb13 → raw model output (scaled): 0.7459
0x000000003853fcedcd0355fec98ca3192833f00b → raw model output (scaled): 0.7142
0x000000003ce0cf2c037493b1dc087204bd7f713e → raw model output (scaled): 0.7530
0x000000007858e6f2668e1e06111cfa24403a5466 → raw model output (scaled): 0.7051
0x00000001a0f57e850c9db68b4a9bc34677437c5c → raw model output (scaled): 0.7051
0x0000000506063a51c6ce59906d8c40f7d7fe92a7 → raw model output (scaled): 0.9006
0x00000029ff545c86524ade7caf132527707948c4 → raw model output (scaled): 0.8758
0x00000087c4cebf

In [9]:
import json

with open("output.json", "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=4))


{
    "0x00000000001accfa9cef68cf5371a23025b6d4b6": 599,
    "0x000000000051d07a4fb3bd10121a343d85818da6": 599,
    "0x000000000096026fb41fc39f9875d164bd82e2dc": 595,
    "0x0000000000e189dd664b9ab08a33c4839953852c": 706,
    "0x0000000002032370b971dabd36d72f3e5a7bf1ee": 643,
    "0x000000000a38444e0a6e37d3b630d7e855a7cb13": 634,
    "0x000000003853fcedcd0355fec98ca3192833f00b": 607,
    "0x000000003ce0cf2c037493b1dc087204bd7f713e": 640,
    "0x000000007858e6f2668e1e06111cfa24403a5466": 599,
    "0x00000001a0f57e850c9db68b4a9bc34677437c5c": 599,
    "0x0000000506063a51c6ce59906d8c40f7d7fe92a7": 765,
    "0x00000029ff545c86524ade7caf132527707948c4": 744,
    "0x00000087c4cebffb95746d1935de7fbcab092f40": 578,
    "0x000000e28faa823d5b53ff6c2922c28335840375": 689,
    "0x000006eee6e39015cb523aebdd4d0b1855aba682": 415,
    "0x00000a9c78912badb92d73b6ed26456e11def5eb": 599,
    "0x00001bc47f0973f794f79c16121cee879e272d6a": 698,
    "0x00002b503a75998c97508916a74fdb41934fa030": 614,
    "0x0